## Simple Mission in Guided-Mode

This notebook serves to get familiar with the simulator

In [1]:
import os

from config import (
    ARDU_LOGS_PATH,
    ARDUPILOT_VEHICLE_PATH,
    ENV_CMD_PYT,
    VEH_PARAMS_PATH,
    BasePort,
)
from helpers import clean, create_process, setup_logging, wait_for_port
from helpers.connections.mavlink.conn import create_tcp_conn
from plan import State
from plan.planner import Plan

clean()

## Launch Copter (ardupilot)

In [2]:
#This must agree with first waypoint in mission.waypoint
spawn_str= '-35.3633245,149.1652241,0.0,0' 

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
sysid = 1
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I{sysid-1} --sysid {sysid} "
    f"--no-rebuild "
    f"--use-dir={ARDU_LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f"--port-offset=0 "  # ArduPilot automatically binds TCP port 5760 (BasePort.ARP)
    f'--custom-location={spawn_str}'
)

create_process(
                vehicle_cmd,
                after="exec bash",
                visible= True,
                suppress_output=False,
                title="ardu_vehicle",
                env_cmd=ENV_CMD_PYT,
            )
wait_for_port(BasePort.ARP, timeout=1)

Waiting for port 5760 to open...


## Connect to the vehicle

In [3]:
conn = create_tcp_conn(
    base_port=BasePort.ARP, 
    offset=0,
    role="client", 
    sysid=sysid
)

print("✅ TCP connection established!")

✅ TCP connection established!


## Create Plan in guided mode

Action commands are sent online, in contrast to auto mode where the full plan is loaded into the UAV before starting the mission.

In [4]:
plan = Plan.square(side_len=5, alt=5, wp_margin=0.5, navegation_speed=5)

In [5]:
plan

🕓 <Plan '📋 Square Trajectory'>
  🕓 <Action '🔧 PREARM'>
    🕓 <Step '🔹 Check disarmed'>
    🕓 <Step '🔹 Check EKF status'>
    🕓 <Step '🔹 Check GPS'>
    🕓 <Step '🔹 Check system'>
  🕓 <Action '⚙️  MODE: GUIDED'>
    🕓 <Step '🔹 Switch to GUIDED'>
  🕓 <Action '🔐 ARM'>
    🕓 <Step '🔹 arm'>
  🕓 <Action '🛫 TAKEOFF'>
    🕓 <Step '🔹 takeoff'>
  🕓 <Action '🛩️ FLY'>
    🕓 <Step '🔹 go to  -> (0, 0, 5)'>
    🕓 <Step '🔹 go to  -> (0, 5, 5)'>
    🕓 <Step '🔹 go to  -> (5, 5, 5)'>
    🕓 <Step '🔹 go to  -> (5, 0, 5)'>
    🕓 <Step '🔹 go to  -> (0, 0, 5)'>
  🕓 <Action '🛬 LAND'>
    🕓 <Step '🔹 land'>

# Execute Plan in guided mode

In [6]:
plan.bind(conn)
setup_logging('plan',verbose=2)
while plan.state != State.DONE:
    plan.act()

17:51:16 - plan - DEBUG - ▶️ Vehicle 1: Plan Started: 📋 Square Trajectory
17:51:16 - plan - DEBUG - ▶️ Vehicle 1: Action Started: 🔧 PREARM
17:51:16 - plan - DEBUG - ▶️ Vehicle 1: Step Started: Check disarmed
17:51:16 - plan - DEBUG - ✅ Vehicle 1: Step Done: Check disarmed
17:51:16 - plan - DEBUG - Vehicle 1: 📡 Requested message EKF_STATUS_REPORT at 1.00 Hz
17:51:16 - plan - DEBUG - ▶️ Vehicle 1: Step Started: Check EKF status
17:51:19 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
17:51:20 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
17:51:21 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
17:51:22 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
17:51:23 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
17:51:24 - plan - DE

In [7]:
print(plan)

✅ <Plan '📋 Square Trajectory'>
  ✅ <Action '🔧 PREARM'>
    ✅ <Step '🔹 Check disarmed'>
    ✅ <Step '🔹 Check EKF status'>
    ✅ <Step '🔹 Check GPS'>
    ✅ <Step '🔹 Check system'>
  ✅ <Action '⚙️  MODE: GUIDED'>
    ✅ <Step '🔹 Switch to GUIDED'>
  ✅ <Action '🔐 ARM'>
    ✅ <Step '🔹 arm'>
  ✅ <Action '🛫 TAKEOFF'>
    ✅ <Step '🔹 takeoff'>
  ✅ <Action '🛩️ FLY'>
    ✅ <Step '🔹 go to  -> (0, 0, 5)'>
    ✅ <Step '🔹 go to  -> (0, 5, 5)'>
    ✅ <Step '🔹 go to  -> (5, 5, 5)'>
    ✅ <Step '🔹 go to  -> (5, 0, 5)'>
    ✅ <Step '🔹 go to  -> (0, 0, 5)'>
  ✅ <Action '🛬 LAND'>
    ✅ <Step '🔹 land'>


In [8]:
plan.reset()

In [9]:
print(plan)

🕓 <Plan '📋 Square Trajectory'>
  🕓 <Action '🔧 PREARM'>
    🕓 <Step '🔹 Check disarmed'>
    🕓 <Step '🔹 Check EKF status'>
    🕓 <Step '🔹 Check GPS'>
    🕓 <Step '🔹 Check system'>
  🕓 <Action '⚙️  MODE: GUIDED'>
    🕓 <Step '🔹 Switch to GUIDED'>
  🕓 <Action '🔐 ARM'>
    🕓 <Step '🔹 arm'>
  🕓 <Action '🛫 TAKEOFF'>
    🕓 <Step '🔹 takeoff'>
  🕓 <Action '🛩️ FLY'>
    🕓 <Step '🔹 go to  -> (0, 0, 5)'>
    🕓 <Step '🔹 go to  -> (0, 5, 5)'>
    🕓 <Step '🔹 go to  -> (5, 5, 5)'>
    🕓 <Step '🔹 go to  -> (5, 0, 5)'>
    🕓 <Step '🔹 go to  -> (0, 0, 5)'>
  🕓 <Action '🛬 LAND'>
    🕓 <Step '🔹 land'>
